In [1]:
import pandas as pd

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sklearn import mixture

In [ ]:
import_file = 'userReviewDates.json'
review = pd.read_json(import_file, lines=True)
review = review.sort_values('user_id')

In [ ]:
review['user_latitude']=pd.Series()
review['user_longitude']=pd.Series()
review['user_city']=pd.Series()
review['user_state']=pd.Series()

In [ ]:
users = review['user_id'].drop_duplicates()
users.index = range(len(users))
users

In [ ]:
# Return array of latitudes and array of longitudes corresponding to given user
def user_locations(user):
    user = [user]
    tmp = review.query('user_id == @user').reset_index(drop=True)
    print('User: %s' % user)
    print('Reviews Included: %d' % len(tmp))
    print('Total Reviews: %d' % tmp['review_count'][0])
    return tmp['latitude'], tmp['longitude']

In [ ]:
import gmplot
user = 'eqWEgMH-DCP74i82BEAZzw'
loc = user_locations(user)
gmap = gmplot.GoogleMapPlotter(36, -150, 16)
gmap.scatter(loc[0], loc[1], 'cornflowerblue', edge_width=10)
gmap.draw("%s.html" % user)

In [ ]:
COORDINATE_DIFFERENCE = 0.4

def same_city(a,b):
    if abs(a[0]-b[0])<COORDINATE_DIFFERENCE and abs(a[1]-b[1])<COORDINATE_DIFFERENCE:
        return True
    else:
        return False

In [ ]:
def estimate_clusters_number(locations):
    i =1 
    while i < len(locations):
        if same_city(locations[0],locations[i]):
            return i
        i = i+1
    return len(locations)

In [ ]:
def gmm(latitudes, longitudes):
    X_train = []
    for i in range(len(latitudes)):
        X_train.append([latitudes[i],longitudes[i]])
    X_train = np.array(X_train)
    
    if len(X_train) > 12:
        components = 12
    else:
        components = len(X_train)
        
    # fit a Gaussian Mixture Model with one component
    clf = mixture.GaussianMixture(n_components=components, covariance_type='full')
    clf.fit(X_train)
    components = estimate_clusters_number(clf.means_)
    clusters = mixture.GaussianMixture(n_components=components, covariance_type='full')  
    clusters.fit(X_train)
    
    return clusters

In [ ]:
def scatterplot(X_train, center, clf):
    # display predicted scores by the model as a contour plot
    x = np.linspace(center[0]-0.2, center[0]+0.2)
    y = np.linspace(center[1]-0.2, center[1]+0.2)
    X, Y = np.meshgrid(x, y)
    XX = np.array([X.ravel(), Y.ravel()]).T
    Z = -clf.score_samples(XX)
    Z = Z.reshape(X.shape)

    CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                     levels=np.logspace(0, 3, 10))
    CB = plt.colorbar(CS, shrink=0.8, extend='both')
    plt.scatter(X_train[:, 0], X_train[:, 1], .9)

    plt.title('Businesses Visited by User')
    plt.axis('tight')
    plt.show()

In [ ]:
import reverse_geocoder as rg
def locate(latitudes, longitudes):
    # create training set
    X_train = []
    for i in range(len(latitudes)):
        X_train.append([latitudes[i],longitudes[i]])
    X_train = np.array(X_train)

    # fit a Gaussian Mixture Model with one component
    clf = gmm(latitudes, longitudes)
    center = clf.means_[0]
    
    # print location
    coordinates = (center[0], center[1])
    results = rg.search(coordinates)[0]    
    #print('User is located at: %s, %s' % (center[0],center[1]))
    #print('%s, %s' % (results['name'],results['admin1']))
    
    # plot users reviews on scatterplot
    #scatterplot(X_train, center, clf)
    
    return center[0], center[1], results['name'], results['admin1']

In [ ]:
def locate_user(user):
    loc = user_locations(user)
    return locate(loc[0], loc[1])

In [ ]:
locations= [] 
for i in range(1000): 
    new = [] 
    loc = locate_user(users[i]) 
    for value in loc: 
        new.append(value) 
        locations.append(new)

In [ ]:
locations

In [ ]:
location_dict = dict(zip(users, locations))

In [ ]:
location_dict['cjpdDjZyprfyDG3RlkVG3w']

In [ ]:
for i in range(1000):
    user = review['user_id'][i]
    review['user_latitude'][i]=location_dict[user][0]
    review['user_longitude'][i]=location_dict[user][1]
    review['user_city'][i]=location_dict[user][2]
    review['user_state'][i]=location_dict[user][3]

In [ ]:
review

In [ ]:
import gmplot
def plot_user(user):
    loc = user_locations(user)
    prediction = location_dict[user]
    latitudes = []
    longitudes = []
    latitudes.append(prediction[0])
    longitudes.append(prediction[1])
    gmap = gmplot.GoogleMapPlotter(latitudes[0], longitudes[0], 13)
    gmap.scatter(loc[0], loc[1], 'cornflowerblue', edge_width=10)
    gmap.scatter(latitudes, longitudes, 'red', edge_width=20)
    gmap.draw("maps/%s.html" % user)

In [ ]:
plot_user('rW8q706dz5-NnXDzMwVkiw')